<a href="https://colab.research.google.com/github/Siliconvalley4uYouthProjects/SpringBoard-Swatcloud/blob/main/Data_Preprocessing_Industry_Labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing libraries

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = pd.read_csv('swe_jobs.csv',header=None,names=['ID','Company','Job Title','Job URL'])
df = df.iloc[1:]
df

,ID,Company,Job Title,Job URL
1,0.0,Amazon,Senior Software Development Engineer,· 4+ years of professional software developmen...
2,1.0,Amazon,Software Development Engineer - Payments,· programming experience with at least one mod...
3,2.0,Amazon,Software Development Engineer - Fintech,bachelor’s degree in computer science or relat...
4,3.0,Amazon,Software Development Engineer,1+ years of experience contributing to the sys...
5,4.0,Amazon,"Embedded Software Development Engineer, Satell...",1+ years of experience contributing to the sys...
...,...,...,...,...
2530,2557.0,Mainz Brady Group,e-Commerce Data Analyst,Microsoft Excel: 1 year (Required)\nData Analy...
2531,2558.0,Brooksource,Data Analyst,"Bachelor's degree in Computer Science, Informa..."
2532,2559.0,Internal Data Resources,Sr. Data Analyst,Work involves conducting detailed analysis of ...
2533,2560.0,Amware Fulfillment,Business Analyst,MS Excel to include advanced features such as ...


In [ ]:
agg = []
for title in df.iloc[:,2]:
  agg.append(title)

In [ ]:
#Counts of different words in the job titles
vectorizer = CountVectorizer(ngram_range = (1,4),stop_words = 'english')
cv = vectorizer.fit_transform(agg)
counts = pd.DataFrame(vectorizer.fit_transform(agg).sum(axis=0),
                      columns=vectorizer.get_feature_names())
counts = counts.T
common_words = counts[0].sort_values(ascending=False)[:50]
print(common_words)

engineer                            1465
software                            1175
software engineer                    567
manager                              497
development                          376
senior                               331
development engineer                 311
software development                 303
ii                                   284
software development engineer        261
engineering                          200
data                                 168
engineer ii                          166
principal                            165
program                              150
google                               144
product                              140
iii                                  138
cloud                                131
program manager                      130
senior software                      128
technical                            121
sr                                   115
product manager                      110
scientist       

In [ ]:
#Labeling the software engineering & tech related jobs

import nltk
industries = ['Software Engineering','Business','Data Analysis','PM']
swe_keywords = ['engineer','software','development','developer']
bus_keywords = ['operations','business']
data_keywords = ['data']
pm_keywords = ['program','product']

ind = []
for i in range(len(df)):
  if len(ind) == len(df):
    break
  try:
    if j == 0:
    #This means that there were no keywords found in the entire job title
      ind.append('Technology (Non-SWE)')
  except:
    pass
  j = 0
  for word in df.iloc[i,2].lower().split(' '):
    word = ''.join(x for x in word if x.isalnum())
    if word in swe_keywords:
      ind.append('Software Engineering')
      j += 1
    elif word in bus_keywords:
      ind.append('Operations')
      j += 1
    elif word in data_keywords:
      ind.append('Data Analysis')
      j += 1
    elif word in pm_keywords:
      ind.append('PM')
      j += 1
    if j != 0:
      break   
df['Industry'] = ind
df['Industry'].value_counts()

Software Engineering    1594
Technology (Non-SWE)     482
PM                       263
Data Analysis            133
Operations                62
Name: Industry, dtype: int64

In [ ]:
df.to_csv('tech_jobs_labeled.csv')

In [57]:
#Labeling the marketing & business related jobs
df = pd.read_csv('marketing_job_descriptions.csv',header=None,names=['Industry','Company','Job Title','Job Description'],skiprows=1)

swe_keywords = ['engineer','software','development','developer']
bus_keywords = ['operations','business']
data_keywords = ['data','analyst']
pm_keywords = ['program','product']
marketing_keywords = ['media','ux','marketing']

ind = []
for i in range(len(df)):
  j = 0
  k = 0
  for word in df.iloc[i,2].lower().split(' '):
    if k == len(df.iloc[i,2].lower().split(' ')):
      #This means that there were no keywords found in the entire job title
      ind.append('Other Business Related')
      break
    word = ''.join(x for x in word if x.isalnum())
    if word in swe_keywords:
      ind.append('Software Engineering')
      j += 1
    elif word in bus_keywords:
      ind.append('Business & Operations')
      j += 1
    elif word in data_keywords:
      ind.append('Data Analysis')
      j += 1
    elif word in marketing_keywords:
      ind.append('Marketing')
      j += 1
    elif word in pm_keywords:
      ind.append('Product/Program Management')
      j += 1
    else:
      k += 1
    if j != 0:
      break
    if k == len(df.iloc[i,2].lower().split(' ')):
      #This means that there were no keywords found in the entire job title
      ind.append('Other Business Related')
      break  
df['Industry'] = ind
df['Industry'].value_counts()

Other Business Related        710
Product/Program Management    259
Business & Operations         186
Marketing                     137
Software Engineering           82
Data Analysis                  50
Name: Industry, dtype: int64

In [60]:
df[df['Industry']=='Other Business Related']['Job Title'].value_counts()

Account Manager                                                             5
Account Manager 1, Inside Sales                                             4
Professional Services II #0000                                              4
Brand Specialist                                                            3
2023 Applied Science Internship - Machine Learning - United States          2
                                                                           ..
Account Executive                                                           1
Vietnam Seller Edu Lead, Asia Seller Education                              1
Account Exec Global Selling KR                                              1
Live Broadcast Manager, CN , Asia Seller Education                          1
2023 J&J Quality & Compliance Organization - Co-Op and Summer Internship    1
Name: Job Title, Length: 671, dtype: int64

In [61]:
df.to_csv('marketing_jobs_labeled.csv')